# Magnetic Models Comparison

In this tutorial, we'll compare various **Magnetic Models**.

## Setup

Let's import the necessary libraries:

In [16]:
import math
import numpy as np
import pandas as pd

%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import Axes3D

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected = True)

In [17]:
import ostk.physics as physics

In [18]:
Scale = physics.time.Scale
Instant = physics.time.Instant
DateTime = physics.time.DateTime
Earth = physics.environment.objects.celestial_bodies.Earth
DipoleModel = physics.environment.magnetic.Dipole
EarthMagneticModel = physics.environment.magnetic.Earth

## Comparison

Let's set up some magnetic models:

In [19]:
magnetic_models = {
    EarthMagneticModel.Type.Dipole: None,
#     EarthMagneticModel.Type.Dipole: DipoleModel(np.array((0, 0, -8.0e15))),
    EarthMagneticModel.Type.EMM2010: None,
    EarthMagneticModel.Type.EMM2015: None,
    EarthMagneticModel.Type.EMM2017: None,
    EarthMagneticModel.Type.IGRF11: None,
    EarthMagneticModel.Type.IGRF12: None,
    EarthMagneticModel.Type.WMM2010: None,
    EarthMagneticModel.Type.WMM2015: None
}

In [20]:
def init_magnetic_models (magnetic_models):
    
    for key in magnetic_models:
        
        if magnetic_models[key] is None:
        
            magnetic_models[key] = EarthMagneticModel(key)

In [21]:
init_magnetic_models(magnetic_models)

In [22]:
instant = Instant.date_time(DateTime(2015, 1, 1, 0, 0, 0), Scale.UTC)

In [23]:
magnetic_models[EarthMagneticModel.Type.Dipole].get_field_value_at(np.array((7000e3, 0.0, 0.0)), instant)

array([[-2.51691725e-06],
       [-3.82878902e-06],
       [ 2.22841012e-05]])

In [24]:
magnetic_models[EarthMagneticModel.Type.EMM2010].get_field_value_at(np.array((7000e3, 0.0, 0.0)), instant)

array([[ 9.75689422e-06],
       [-2.11723158e-06],
       [ 2.04720913e-05]])

In [25]:
def calc_B_2d (magnetic_model, X, Z):
    
    size_x = len(X)
    size_z = len(X[0])
    
    size = size_x * size_z
    
    Bx = np.zeros(size)
    Bz = np.zeros(size)
    
    earth_radius_m = Earth.equatorial_radius.in_meters()
    
    i = 0
    
    for x, z in np.vstack([X.ravel(), Z.ravel()]).T:
        
        if math.sqrt(pow(abs(x), 2) + pow(abs(z), 2)) < earth_radius_m:
            
            Bx[i] = 0.0
            Bz[i] = 0.0
            
        else:

            try:

                b = magnetic_model.get_field_value_at(np.array((x, 0.0, z)), instant).T[0]

                bx = b[0]
                bz = b[2]
            
                Bx[i] = bx
                Bz[i] = bz

            except Exception as e:

                print(e)

                print('x = {}'.format(x))
                print('z = {}'.format(z))
                
                raise e
            
        i += 1
        
    return Bx.reshape(size_x, size_z), \
           Bz.reshape(size_x, size_z)

In [26]:
def calc_B_3d (magnetic_model, X, Y, Z):
    
    size_x = len(X)
    size_y = len(X[0])
    size_z = len(X[0][0])
    
    size = size_x * size_y * size_z
    
    Bx = np.zeros(size)
    By = np.zeros(size)
    Bz = np.zeros(size)
    
    earth_radius_m = Earth.equatorial_radius.in_meters()
    
    i = 0
    
    for x, y, z in np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T:
        
        if math.sqrt(pow(abs(x), 2) + pow(abs(y), 2) + pow(abs(z), 2)) < earth_radius_m:
            
            Bx[i] = 0.0
            By[i] = 0.0
            Bz[i] = 0.0
            
        else:

            try:

                b = magnetic_model.get_field_value_at(np.array((x, y, z)), instant).T[0]

                bx = b[0]
                by = b[1]
                bz = b[2]

                Bx[i] = bx
                By[i] = by
                Bz[i] = bz

            except Exception as e:

                print(e)

                print('x = {}'.format(x))
                print('y = {}'.format(y))
                print('z = {}'.format(z))
                
                raise e
            
        i += 1
        
    return Bx.reshape(size_x, size_y, size_z), \
           By.reshape(size_x, size_y, size_z), \
           Bz.reshape(size_x, size_y, size_z)

In [27]:
lim = 100000e3

xlim = (-lim, +lim)
zlim = (-lim, +lim)

nx, nz = 64, 64

x = np.linspace(xlim[0], xlim[1], nx)
z = np.linspace(zlim[0], zlim[1], nz)

X, Z = np.meshgrid(x, z)

Bx, Bz = calc_B_2d(magnetic_models[EarthMagneticModel.Type.EMM2015], X, Z)

In [28]:
fig = plt.figure()

ax = fig.add_subplot(111)

# Plot Earth circle

ax.add_artist(Circle((0.0, 0.0), Earth.equatorial_radius.in_meters(), color = 'b'))

# Plot B

color = 2 * np.log(np.hypot(Bx, Bz))

ax.streamplot(x, z, Bx, Bz, color=color, linewidth=1, cmap=plt.cm.inferno,
              density=2, arrowstyle='->', arrowsize=1.5)

ax.set_xlabel('$x_{ITRF}\ [m]$')
ax.set_ylabel('$z_{ITRF}\ [m]$')
ax.set_xlim(xlim[0], xlim[1])
ax.set_ylim(zlim[0], zlim[1])
ax.set_aspect('equal')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning:

divide by zero encountered in log



In [29]:
lim = 10000e3

xlim = (-lim, +lim)
ylim = (-lim, +lim)
zlim = (-lim, +lim)

nx, ny, nz = 9, 9, 9

x = np.linspace(xlim[0], xlim[1], nx)
y = np.linspace(ylim[0], ylim[1], ny)
z = np.linspace(zlim[0], zlim[1], nz)

X, Y, Z = np.meshgrid(x, y, z)

Bx, By, Bz = calc_B_3d(magnetic_models[EarthMagneticModel.Type.EMM2010], X, Y, Z)

# print('X = ', X)
# print('Y = ', Y)
# print('Z = ', Z)

# print('Bx = ', Bx)
# print('By = ', By)
# print('Bz = ', Bz)

In [32]:
fig = plt.figure()

ax = fig.gca(projection = '3d')

# Plot Earth sphere

u = np.linspace(0.0, 2.0 * np.pi, 100)
v = np.linspace(0.0, np.pi, 100)

earth_radius_m = float(Earth.equatorial_radius.in_meters())

x = earth_radius_m * np.outer(np.cos(u), np.sin(v))
y = earth_radius_m * np.outer(np.sin(u), np.sin(v))
z = earth_radius_m * np.outer(np.ones(np.size(u)), np.cos(v))

ax.plot_surface(x, y, z,  rstride = 4, cstride = 4, color = 'b', linewidth = 0, alpha = 1)

# Plot B

ax.quiver(X, Y, Z, Bx, By, Bz, length = 1e11, color = 'g')

ax.set_xlabel('$x_{ITRF}\ [m]$')
ax.set_ylabel('$y_{ITRF}\ [m]$')
ax.set_zlabel('$z_{ITRF}\ [m]$')
ax.set_xlim(xlim[0], xlim[1])
ax.set_ylim(ylim[0], ylim[1])
ax.set_zlim(zlim[0], zlim[1])
# ax.set_aspect('equal')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---